In [1]:
import pandas as pd
import numpy as np
import nltk, itertools
from gensim.models import Word2Vec
import nltk, os
import tensorflow as tf
import itertools
import random

In [2]:
#taking the original dataset:

file_data = open('gh_create_all_fields_cleaned_data.txt', 'rt')
# reading the sentences:
sentences=file_data.read()
sentences=sentences.split('\n')



In [15]:
#importing randomly selected 50 sentenes from the OG dataset:
import random
train_sent=[]

for i in range(11):
    num = random.randrange(0, len(sentences))
    train_sent.append(sentences[num])
    
# train_sent.append("$& $$& i love $pence is $$&")



In [16]:
 out, inp = [" ".join(x.split()[1:]) for x in train_sent], [" ".join(x.split()[:1]) for x in train_sent]

In [17]:
table_map=dict()
out_map=dict()
for index, val in enumerate(out):
    table_map[inp[index]]=out[index]
    out_map[out[index]]=inp[index]
train_sent=[]
for key in table_map:
    sent="$"+key+"&"+" "+table_map[key]
    train_sent.append(sent)

In [18]:
print(len(train_sent))

10


In [26]:
# importing the essential visualization parameters:
from d2l import tensorflow as d2l
import math
import numpy as np
import tensorflow as tf

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

train_random_iter, vocab_random_iter = d2l.load_data_time_machine(
    batch_size, num_steps, use_random_iter=True)



char_data =""

for sent in train_sent:
    char_data+=sent+"\n"

chars= list(set(char_data))

#create dictionary mapping for each char
char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
ix_to_char = {i:ch for (i,ch) in enumerate(chars)}

In [27]:
def get_params2(vocab_size, num_hidden):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32)

    # Hidden layer parameters
    E = tf.Variable(normal((vocab_size, 10)), dtype=tf.float32)
    W_xh = tf.Variable(normal((num_inputs, num_hiddens)), dtype=tf.float32)
    W_hh = tf.Variable(normal((num_hiddens, num_hiddens)), dtype=tf.float32)
    b_h = tf.Variable(tf.zeros(num_hiddens), dtype=tf.float32)
    # Output layer parameters
    W_hq = tf.Variable(normal((num_hiddens, num_outputs)), dtype=tf.float32)
    b_q = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32)
    M = tf.Variable(tf.random.uniform([W_xh.shape[1], vocab_size], dtype=tf.float32))
    params = [W_xh, W_hh, b_h, W_hq, b_q, E, M]
#     params = [W_xh, W_hh, b_h, W_hq, b_q]
    return params

In [66]:
# function for converting numpy vectors into tensorflow vectors:
def my_func(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    print(arg)
    return arg

In [67]:
def init_rnn_state2(batch_size, num_hiddens):
    return (tf.zeros((batch_size, num_hiddens)), )

In [68]:
def rnn2(inputs, state, params, curr_sent, vocab_size, word2vec_model):
    # Here `inputs` shape: (`num_steps`, `batch_size`, `vocab_size`)
    W_xh, W_hh, b_h, W_hq, b_q,E, M = params
    
    # calculating the values from the encoder function
    result=tf.zeros([vocab_size, 1], tf.float32)
    for every_char in curr_sent:
        tfcharEmb = my_func(word2vec_model[every_char])
        tfcharEmb=tf.reshape(tfcharEmb, [10, 1])
        result+= tf.matmul(E, tfcharEmb)
        
#     M=tf.random.uniform([W_xh.shape[1], vocab_size], dtype=tf.float32)
    H, = state
    outputs = []


    for X in inputs:
        X = tf.reshape(X,[-1,W_xh.shape[0]])
        a= tf.matmul(X, W_xh)
        b= tf.matmul(H, W_hh)
        c = tf.matmul(M, result)
        c =  tf.reshape(c, [c.shape[1], c.shape[0]])
        H = tf.nn.sigmoid(a + b + c + b_h, name='sigmoid')
        Y = tf.matmul(H, W_hq) + b_q
        y_out = tf.nn.softmax(Y)
        outputs.append(y_out)
    return tf.concat(outputs, axis=0), (H,)

In [69]:
class RNNModelScratch2: #@save
    """A RNN Model implemented from scratch."""
    def __init__(self, vocab_size, num_hiddens,
                 init_state, forward_fn, word2vec):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.init_state, self.forward_fn = init_state, forward_fn
        self.word2vec=word2vec

    def __call__(self, X, state, params, curr_sent):
        print(X, X.shape)
#         X = tf.one_hot(tf.transpose(X), self.vocab_size)
#         X = tf.cast(X, tf.float32)
        return self.forward_fn(X, state, params, curr_sent, self.vocab_size, self.word2vec)

    def begin_state(self, batch_size):
        return self.init_state (batch_size, self.num_hiddens)
    

In [205]:
def train(model, params, table_map, char_to_ix, ix_to_char, inp, vocab_size):
    "traing the modified model"
    state=None
    
    threshold = 0.05
    lr=1
    loss = tf.keras.losses.CategoricalCrossentropy()
    updater = tf.keras.optimizers.SGD(lr)
    optimizer = tf.keras.optimizers.Adam()
    loss_res=[] # storage for the resultant loss
    loss_epochs = [] #keeping of the loss per epoch
    epochs = 1200
    metric = d2l.Accumulator(2)
    timer=d2l.Timer()
    
    
    while epochs>0:
        
        loss_per_epoch=0
        
        for batch in range(0, len(inp), 2):
            currBatch_X=[]
            currBatch_y=[]

            maxLen = len(max(inp[batch:batch+2], key=len))
            maxLen+=2
            
            
            for every_table in inp[batch:batch+2]:

                #padding the current word
                target_word = "$"+every_table+"&"
    #             print("target word -> ",target_word)

                input_str = target_word[0:len(target_word)-1]
                target_str = target_word[1:len(target_word)]

                inputs = [char_to_ix[ch] for ch in input_str]
                targets = [char_to_ix[ch] for ch in target_str]
                print(inputs)
                X = tf.reshape(inputs, (1, len(inputs)))
                Y_out = tf.reshape(targets, (1, len(targets)))
                
                X = tf.one_hot(tf.transpose(X), vocab_size)
              
                
                for _ in range(maxLen-len(target_word)):
                    temp = [0] * vocab_size
#                     temp = my_func(temp)
                    temp = tf.Variable(tf.reshape(temp, [-1, len(temp)]))
                    temp = tf.reshape(temp, (1, temp.shape[0], temp.shape[1]))
                    temp =tf.cast(temp, tf.float32)
                    X = tf.concat([X, temp],0)
                
                
                print(" X-> ", X)
                X = tf.cast(X, tf.float32)
                
                print(X.shape)
                
                
                Y_true=tf.one_hot(Y_out, len(char_to_ix),on_value=1.0, off_value=0.0,axis=-1)
                Y_true=Y_true[0];


                # The actual crux of training the data:

    #             if state is None:

    #                 state = model.begin_state(batch_size=X.shape[0])

    #             with tf.GradientTape(persistent=True) as g:
    # #                 print('I am here 1')
    #                 g.watch(params)
    #                 y_hat, state= model(X, state, params, table_map[every_table])
    #                 y = tf.reshape(Y_out, (-1))
    #                 axis_0 = np.shape(y_hat.numpy())[0]
    #                 axis_1 = np.shape(y_hat.numpy())[1]
    #                 Y_true=tf.reshape(Y_true, [axis_0, axis_1])
    #                 # calculating the loss
    #                 l = loss(Y_true, y_hat)

    # #             print(l)

    # #             print('I am here 2')
    #             grads = g.gradient(l, params)
    #             grads = grad_clipping(grads, 1)
    #             optimizer.apply_gradients(zip(grads, params))
    # #             updater.apply_gradients(zip(grads, params))
    # #             print('I am here 3')
    # #             metric.add(l * tf.size(y).numpy(), tf.size(y).numpy())
    #             loss_per_epoch=l

    #         # after every epoch
    #         loss_epochs.append(loss_per_epoch)
    # #         loss_res.append(math.exp(metric[0] / metric[1]))
        epochs=epochs-1
        break
    return loss_epochs

In [206]:
def grad_clipping(grads, theta): #@save
    """Clip the gradient."""
    theta = tf.constant(theta, dtype=tf.float32)
    norm = tf.math.sqrt(sum((tf.reduce_sum(grad ** 2)).numpy()
                        for grad in grads))
    norm = tf.cast(norm, tf.float32)
    new_grad = []
    if tf.greater(norm, theta):
        for grad in grads:
            new_grad.append(grad * theta / norm)
    else:
        for grad in grads:
            new_grad.append(grad)
    return new_grad

In [207]:
# functions to calculate word tokenizations and tokenizers:
# which would be later used for calculating the word to 

def filter_words(sentence):
    ll = [[nltk.word_tokenize(w), ' '] for w in sentence.split()]
#     print(list(itertools.chain(*list(itertools.chain(*ll)))))
#     return [word.lower() if word.isalpha() else " " for word in [nltk.word_tokenize(sentence), " "]]
    return list(itertools.chain(*list(itertools.chain(*ll))))

def tokenize(text):
    return [filter_words(sentence) for sentence in nltk.sent_tokenize(text)]

tokenized_words=[]
sent_data=[]
for sentence in out:
    sent_mod=''
    for char in sentence:
        sent_mod+=char+' ' 
#     print(sent_mod)
#     print(nltk.sent_tokenize(sent_mod))
    tokenized_words.extend(tokenize(sent_mod))
    sent_data.append(sentence)

word2vec_model = Word2Vec(tokenized_words, 
                          sg=1,  # here we will use skipgram 0 is for CBROW; skip gram is for smaller models 
                          size=10,
                          min_count=1 # word needs to occur atleast once
                         )
 

In [208]:
num_hiddens = 512        # declaration of number of hidden layers
lr=0.1                   # declaration of regularization parameter
seq_len=10               # this is a sequence length
updater = tf.keras.optimizers.SGD(lr)
# data_reader = DataReader("temp.txt", seq_len)
#defining the required params:
params = get_params2(len(chars), num_hiddens)

model=RNNModelScratch2(len(chars), num_hiddens, init_rnn_state2, rnn2, word2vec_model)
loss_res =train(model, params, table_map, char_to_ix, ix_to_char, inp, len(chars))

[16, 1, 7, 15, 19, 12, 14, 0, 15]
 X->  tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]], shape=(9, 1, 29), dtype=float32)
(9, 1, 29)
[16, 21, 7, 25, 17, 15]
 X->  tf.Tensor(

temp = [0]*29

In [178]:
temp = [0] * 29
#                     temp = my_func(temp)
temp = tf.Variable(tf.reshape(temp, [-1, len(temp)]))

In [180]:
temp=tf.Variable(temp)

In [184]:
tf.reshape(temp, (1,1,29))

<tf.Tensor: shape=(1, 1, 29), dtype=int32, numpy=
array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]], dtype=int32)>

In [215]:
x = 2 
y=2
for X, Y in train_iter:
    x =X 
    y=Y
    break

In [211]:
x

<tf.Tensor: shape=(32, 35), dtype=int32, numpy=
array([[ 4, 15,  9, ..., 22,  2, 12],
       [ 1, 21, 14, ...,  5,  6,  1],
       [ 6,  1, 16, ..., 19,  4, 11],
       ...,
       [ 2, 10,  1, ...,  7, 10,  2],
       [ 5,  6,  2, ...,  2, 24, 20],
       [ 8, 13,  3, ...,  3,  1, 11]], dtype=int32)>

In [216]:
X = tf.one_hot(tf.transpose(x), len(vocab))
X = tf.cast(X, tf.float32)

In [217]:
X

<tf.Tensor: shape=(35, 32, 28), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 

In [218]:
y

<tf.Tensor: shape=(32, 35), dtype=int32, numpy=
array([[15,  9,  5, ...,  2, 12, 12],
       [21, 14, 21, ...,  6,  1,  7],
       [ 1, 16,  7, ...,  4, 11, 13],
       ...,
       [10,  1,  9, ..., 10,  2,  1],
       [ 6,  2,  1, ..., 24, 20, 12],
       [13,  3,  9, ...,  1, 11,  7]], dtype=int32)>